In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from bill_pre import *
import datetime

## 读取账单信息并进行预处理

- 列补充
    - 单独做出年、月、日、时、相对首日的日数这几个列
    - 所有条目按照时间排序
    - 将收入支出合并到金额条目中。为了方便分析，支出为正，收入为负
    - 计算当时总余额
        - （先设置初始为0，然后看总账单余额最低点，再将余额最低点设为0就可以得到近似的余额）
    
- 需要人工
    - 去除付款后退款的条目（相当于交易未发生）：主要是淘宝和携程
    - 合并微信转出后到支付宝花掉的条目
    - 去掉氪金的项目别被发现了
    - 标出自动续费
    - 对商家进行去重
    - 三级标注：项目 - 类别 - 商家

In [7]:
time_ref = pd.Timestamp('2024-06-30 00:00:00')  # 相对的起始日子
money_left = 295.23  # 目前剩余的钱

In [19]:
bill = pd.read_excel(r'F:\today\对账\bill_ok.xlsx')
bill = bill[bill['账户'] == '生活费']
bill = bill[['类型', '金额', '项目', '分类', '商家', '日期']]
bill = bill.fillna('其它', inplace=False)
bill.columns = ['类型', '总金额', '项目', '分类', '商家', '日期']  # 总金额代表有符号方向的金额，金额改为绝对金额
bill['金额'] = bill['总金额'].abs()
bill.head()

,类型,总金额,项目,分类,商家,日期,金额
0,支出,-14.30,日常,晚饭,炒饭,2024-07-25 17:05:00,14.30
1,支出,-2.00,日常,交通,公交,2024-07-25 16:30:00,2.00
2,支出,-5.06,享受,奶茶,小卖部,2024-07-25 12:56:00,5.06
3,支出,-14.00,日常,中饭,食堂,2024-07-25 11:51:00,14.00
4,收入,2.80,日常收入,退款收入,其它收入,2024-07-25 11:08:00,2.80


In [20]:
# 单独做出年、月、日、时、相对首日的日数这几个列
week_dict = {0:"一", 1:"二", 2:"三", 3:"四", 4:"五", 5:"六", 6:"日"}
def get_ymdh(time_stamp, time_ref):
    """
    输入单个时间戳，返回、年、月、日、时、相对首日日数
    ---
    输入
    - time_stamp: 单个时间戳
    - time_ref: 参考时间, 也是一个时间戳
    """
    year = time_stamp.year
    month = time_stamp.month
    day = time_stamp.day
    day_w = "星期"+week_dict[time_stamp.day_of_week]  # 0代表周一，6代表周日
    hour = time_stamp.hour
    week = time_stamp.week
    
    time_delta = time_stamp - time_ref
    day_delta = time_delta.days
    daymonth_delta = day + hour / 24 + time_stamp.minute / 1440
    dayweek_delta = time_stamp.day_of_week + hour / 24 + time_stamp.minute / 1440

    hour_delta = hour + time_stamp.minute/60
    return (year, month, week, day, 
            day_w, hour, day_delta, 
            hour_delta, daymonth_delta, dayweek_delta)


(bill['年'], bill['月'], bill['周'], bill['日'], 
 bill['星期'], bill['时'], bill['相对日'], 
 bill['相对时'], bill["相对月内日"], bill['相对星期天']) = zip(*bill['日期'].apply(lambda x: get_ymdh(x, time_ref)))

In [21]:
# 按时间排序
bill = bill.sort_index(ascending=True)
bill['temp_amont'] = - bill['总金额']
bill['余额'] = [money_left] + (bill['temp_amont'].cumsum() + money_left).tolist()[:-1]
bill.sort_index(ascending=False)
bill = bill.drop(["temp_amont"], axis=1)
bill.to_excel(r'F:\today\对账\bill_ok_proc.xlsx', index=False)

In [40]:
date_min_choose = pd.Timestamp("2024-07-01")

In [45]:
pd.Timestamp("2024-06-30").week

26

test

In [22]:
bill_ori = pd.read_excel('bill_ok_proc.xlsx')

date_min_choose = pd.Timestamp("2024-07-01")
date_max_choose = pd.Timestamp("2024-07-20")

bill = bill_ori[(bill_ori['日期'] >= date_min_choose) & (bill_ori['日期'] <= date_max_choose)]

bill.loc[:,"相对日"] = bill["相对日"] - bill["相对日"].min()

bill = bill.sort_values(by="日期", ascending=True)

for col in ['项目', '分类', '商家']:
    name_add_now = col + "累计金额"
    for item in set(bill[col].values):
        bool_sel = bill[col] == item
        cum_sel = bill[bool_sel]["金额"].cumsum()
        bill.loc[cum_sel.index, name_add_now] = cum_sel.values
col = "类型"
name_add_now = "累计金额"
for item in set(bill[col].values):
    bool_sel = bill[col] == item
    cum_sel = bill[bool_sel]["金额"].cumsum()
    bill.loc[cum_sel.index, name_add_now] = cum_sel.values

In [2]:
import plotly.express as px
import plotly.graph_objects as go

In [38]:
bill.pivot_table(
    values='金额', 
    index='时', 
    columns='项目', 
    aggfunc='sum'
).fillna(0)

项目,万达,享受,健身,屋の改造,工作,恋爱,日常,生活费收入,租房押金回款收入
时,,,,,,,,,
0,0.0,183.00,0.0,0.0,0.00,0.00,37.80,0.0,0.0
1,0.0,198.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0
2,0.0,30.00,0.0,8.9,0.00,0.00,0.00,0.0,0.0
7,0.0,0.00,0.0,0.0,0.00,0.00,29.40,0.0,0.0
8,0.0,0.00,0.0,0.0,0.00,0.00,16.60,0.0,0.0
9,0.0,0.00,0.0,0.0,0.00,0.00,27.70,0.0,0.0
10,0.0,0.00,0.0,0.0,39.00,0.00,45.08,0.0,0.0
11,0.0,5.40,0.0,0.0,0.00,34.89,192.72,0.0,0.0
12,0.0,26.82,0.0,0.0,0.00,71.44,0.00,0.0,0.0


In [4]:
exp_data

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [62]:
pix_df = bill.pivot_table(
    index="项目", values="金额", aggfunc="sum"
).reset_index(inplace=False).sort_values(by="金额", ascending=False)
pix_df

,项目,金额
6,日常,1089.56
7,生活费收入,1000.00
1,享受,808.29
8,租房押金回款收入,565.00
5,恋爱,213.36
4,工作,86.15
0,万达,73.00
2,健身,11.00
3,屋の改造,8.90


In [65]:
px.bar(pix_df, x='金额', y='项目', orientation='h')